<a href="https://colab.research.google.com/github/anvith1995/InvestQode-Data-Analyst/blob/main/InvestCode_Data_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import duckdb
import os


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sunnysai12345/nse-future-and-options-dataset-3m")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'nse-future-and-options-dataset-3m' dataset.
Path to dataset files: /kaggle/input/nse-future-and-options-dataset-3m


In [ ]:
files = os.listdir(path)
print(files)

raw_data = pd.read_csv(os.path.join(path, files[0]))
raw_data['exchange_id'] = 1
raw_data.head()

['3mfanddo.csv']


,Unnamed: 0,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,exchange_id
0,160393,FUTIDX,BANKNIFTY,29-Aug-2019,0.0,XX,28805.65,28924.00,28140.55,28499.30,28499.30,214569.0,1225914.96,1675780.0,234640.0,01-AUG-2019,1
1,160394,FUTIDX,BANKNIFTY,26-Sep-2019,0.0,XX,28926.40,29030.55,28251.70,28611.45,28611.45,2484.0,14245.95,51400.0,-80.0,01-AUG-2019,1
2,160395,FUTIDX,BANKNIFTY,31-Oct-2019,0.0,XX,29000.00,29105.00,28355.55,28699.05,28699.05,598.0,3434.43,9460.0,4860.0,01-AUG-2019,1
3,160396,FUTIDX,NIFTY,29-Aug-2019,0.0,XX,11098.40,11098.40,10901.10,11015.35,11015.35,199881.0,1650955.24,19001400.0,1339200.0,01-AUG-2019,1
4,160397,FUTIDX,NIFTY,26-Sep-2019,0.0,XX,11136.35,11145.20,10955.00,11066.60,11066.60,5283.0,43841.57,893625.0,66750.0,01-AUG-2019,1


In [ ]:
raw_data = raw_data.drop(columns=["Unnamed: 0"])

In [ ]:
# Connect to DuckDB (in-memory database)
con = duckdb.connect(':memory:')

# Load raw data into DuckDB
con.sql(f"""
    CREATE TABLE raw_data AS
    SELECT * FROM raw_data
""")

print("Raw data loaded successfully!")
print(f"Total rows: {con.execute('SELECT COUNT(*) FROM raw_data').fetchone()[0]}")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Raw data loaded successfully!
Total rows: 2533210


In [ ]:
con.sql("""CREATE TABLE EXCHANGE (
    exchange_id INTEGER PRIMARY KEY,
    exchange_code VARCHAR UNIQUE,
    exchange_name VARCHAR
); """)


In [ ]:
con.sql("""
INSERT INTO EXCHANGE VALUES
(1,'NSE','National Stock Exchange')
""")

In [ ]:
con.sql("""Select * from EXCHANGE """)

┌─────────────┬───────────────┬─────────────────────────┐
│ exchange_id │ exchange_code │      exchange_name      │
│    int32    │    varchar    │         varchar         │
├─────────────┼───────────────┼─────────────────────────┤
│           1 │ NSE           │ National Stock Exchange │
└─────────────┴───────────────┴─────────────────────────┘

In [ ]:
con.sql("""
CREATE TABLE INSTRUMENTS (
    instrument_id INTEGER PRIMARY KEY,
    symbol VARCHAR,
    instrument_type VARCHAR,
); """ )


In [ ]:
#con.execute("DROP TABLE INSTRUMENTS")

In [ ]:
con.sql("""
INSERT INTO INSTRUMENTS
SELECT
    ROW_NUMBER() OVER (ORDER BY symbol, instrument_type) AS instrument_id,
    symbol,
    instrument_type,
FROM (
    SELECT DISTINCT
        SYMBOL AS symbol,
        INSTRUMENT AS instrument_type
    FROM raw_data
) t;

""")


In [ ]:
con.sql(""" Select * from INSTRUMENTS""")

┌───────────────┬────────────┬─────────────────┐
│ instrument_id │   symbol   │ instrument_type │
│     int32     │  varchar   │     varchar     │
├───────────────┼────────────┼─────────────────┤
│             1 │ ACC        │ FUTSTK          │
│             2 │ ACC        │ OPTSTK          │
│             3 │ ADANIENT   │ FUTSTK          │
│             4 │ ADANIENT   │ OPTSTK          │
│             5 │ ADANIPORTS │ FUTSTK          │
│             6 │ ADANIPORTS │ OPTSTK          │
│             7 │ ADANIPOWER │ FUTSTK          │
│             8 │ ADANIPOWER │ OPTSTK          │
│             9 │ AMARAJABAT │ FUTSTK          │
│            10 │ AMARAJABAT │ OPTSTK          │
│             · │  ·         │   ·             │
│             · │  ·         │   ·             │
│             · │  ·         │   ·             │
│           319 │ VEDL       │ FUTSTK          │
│           320 │ VEDL       │ OPTSTK          │
│           321 │ VOLTAS     │ FUTSTK          │
│           322 │ VO

In [ ]:
con.sql("""
CREATE TABLE CONTRACTS (
    contract_id INTEGER PRIMARY KEY,
    instrument_id INTEGER,
    expiry_date DATE,
    strike_price DOUBLE,
    option_type VARCHAR,
    UNIQUE (instrument_id, expiry_date, strike_price, option_type)
);

""")


In [ ]:
con.execute("""
INSERT INTO CONTRACTS
SELECT
    ROW_NUMBER() OVER (
        ORDER BY instrument_id, expiry_date, strike_price, option_type
    ) AS contract_id,
    instrument_id,
    expiry_date,
    strike_price,
    option_type
FROM (
    SELECT DISTINCT
        i.instrument_id,
        STRPTIME(r.EXPIRY_DT, '%d-%b-%Y')::DATE AS expiry_date,
        r.STRIKE_PR AS strike_price,
        r.OPTION_TYP AS option_type
    FROM raw_data r
    JOIN INSTRUMENTS i
      ON r.SYMBOL = i.symbol
     AND r.INSTRUMENT = i.instrument_type
) t;

""")


In [ ]:
con.sql("""Select * from raw_data""")

┌────────────┬────────────┬─────────────┬───────────┬────────────┬──────────┬──────────┬──────────┬──────────┬───────────┬───────────┬────────────┬────────────┬───────────┬─────────────┬─────────────┐
│ INSTRUMENT │   SYMBOL   │  EXPIRY_DT  │ STRIKE_PR │ OPTION_TYP │   OPEN   │   HIGH   │   LOW    │  CLOSE   │ SETTLE_PR │ CONTRACTS │ VAL_INLAKH │  OPEN_INT  │ CHG_IN_OI │  TIMESTAMP  │ exchange_id │
│  varchar   │  varchar   │   varchar   │  double   │  varchar   │  double  │  double  │  double  │  double  │  double   │  double   │   double   │   double   │  double   │   varchar   │    int64    │
├────────────┼────────────┼─────────────┼───────────┼────────────┼──────────┼──────────┼──────────┼──────────┼───────────┼───────────┼────────────┼────────────┼───────────┼─────────────┼─────────────┤
│ FUTIDX     │ BANKNIFTY  │ 29-Aug-2019 │       0.0 │ XX         │ 28805.65 │  28924.0 │ 28140.55 │  28499.3 │   28499.3 │  214569.0 │ 1225914.96 │  1675780.0 │  234640.0 │ 01-AUG-2019 │          

In [ ]:
con.sql(""" Select  i.instrument_id , i.instrument_type, i.symbol, c.expiry_date,c.option_type
from INSTRUMENTS i
left join CONTRACTS c on i.instrument_id  = c.instrument_id
where symbol = 'ACC' and expiry_date = '2019-08-29'
and INSTRUMENT_TYPE = 'FUTSTK' """)

┌───────────────┬─────────────────┬─────────┬─────────────┬─────────────┐
│ instrument_id │ instrument_type │ symbol  │ expiry_date │ option_type │
│     int32     │     varchar     │ varchar │    date     │   varchar   │
├───────────────┼─────────────────┼─────────┼─────────────┼─────────────┤
│             1 │ FUTSTK          │ ACC     │ 2019-08-29  │ XX          │
└───────────────┴─────────────────┴─────────┴─────────────┴─────────────┘

In [ ]:
con.sql("""
    SELECT *
    FROM raw_data
    WHERE INSTRUMENT = 'FUTSTK'
    AND SYMBOL = 'ACC'
    AND EXPIRY_DT = '29-Aug-2019'
""")

┌────────────┬─────────┬─────────────┬───────────┬────────────┬─────────┬─────────┬─────────┬─────────┬───────────┬───────────┬────────────┬───────────┬───────────┬─────────────┬─────────────┐
│ INSTRUMENT │ SYMBOL  │  EXPIRY_DT  │ STRIKE_PR │ OPTION_TYP │  OPEN   │  HIGH   │   LOW   │  CLOSE  │ SETTLE_PR │ CONTRACTS │ VAL_INLAKH │ OPEN_INT  │ CHG_IN_OI │  TIMESTAMP  │ exchange_id │
│  varchar   │ varchar │   varchar   │  double   │  varchar   │ double  │ double  │ double  │ double  │  double   │  double   │   double   │  double   │  double   │   varchar   │    int64    │
├────────────┼─────────┼─────────────┼───────────┼────────────┼─────────┼─────────┼─────────┼─────────┼───────────┼───────────┼────────────┼───────────┼───────────┼─────────────┼─────────────┤
│ FUTSTK     │ ACC     │ 29-Aug-2019 │       0.0 │ XX         │  1547.1 │  1555.4 │  1506.4 │ 1522.65 │   1522.65 │    3536.0 │   21691.65 │ 2292000.0 │   63200.0 │ 01-AUG-2019 │           1 │
│ FUTSTK     │ ACC     │ 29-Aug-201

In [ ]:
con.sql("""
CREATE TABLE fact_trade (
    trade_id BIGINT PRIMARY KEY,
    contract_id INTEGER,
    exchange_id INTEGER,
    trade_date DATE,

    open_price DOUBLE,
    high_price DOUBLE,
    low_price DOUBLE,
    close_price DOUBLE,
    settle_price DOUBLE,

    volume BIGINT,
    turnover DOUBLE,
    open_interest BIGINT,
    change_in_oi BIGINT
);
""")


In [ ]:
con.execute("""
INSERT INTO fact_trade
SELECT
    ROW_NUMBER() OVER () AS trade_id,
    c.contract_id,
    1 AS exchange_id,
    STRPTIME(r.TIMESTAMP, '%d-%b-%Y')::DATE,

    r.OPEN,
    r.HIGH,
    r.LOW,
    r.CLOSE,
    r.SETTLE_PR,

    r.CONTRACTS,
    r.VAL_INLAKH,
    r.OPEN_INT,
    r.CHG_IN_OI
FROM raw_data r
JOIN INSTRUMENTS i
  ON r.SYMBOL = i.symbol
 AND r.INSTRUMENT = i.instrument_type
JOIN CONTRACTS c
  ON c.instrument_id = i.instrument_id
 AND STRPTIME(r.EXPIRY_DT, '%d-%b-%Y')::DATE = c.expiry_date
 AND COALESCE(r.STRIKE_PR,0) = COALESCE(c.strike_price,0)
 AND COALESCE(r.OPTION_TYP,'NA') = COALESCE(c.option_type,'NA');
""")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql(""" Select count(*) from fact_trade""")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      2533210 │
└──────────────┘

In [ ]:
# List Of Tables
# EXCHANGE
# INSTRUMENTS
# CONTRACTS
# fact_trade



In [ ]:
con.sql("""
Select f.*,
e.exchange_name,
c.instrument_id ,
c.expiry_date ,
c.strike_price,
c.option_type,
i.instrument_type,
i.symbol
from fact_trade f
left join exchange e on f.exchange_id =e.exchange_id
left join contracts c on c.contract_id = f.contract_id
left join instruments i on i.instrument_id = c.instrument_id
where change_in_oi > 2000000 """)

┌──────────┬─────────────┬─────────────┬────────────┬────────────┬────────────┬───────────┬─────────────┬──────────────┬────────┬────────────┬───────────────┬──────────────┬─────────────────────────┬───────────────┬─────────────┬──────────────┬─────────────┬─────────────────┬────────────┐
│ trade_id │ contract_id │ exchange_id │ trade_date │ open_price │ high_price │ low_price │ close_price │ settle_price │ volume │  turnover  │ open_interest │ change_in_oi │      exchange_name      │ instrument_id │ expiry_date │ strike_price │ option_type │ instrument_type │   symbol   │
│  int64   │    int32    │    int32    │    date    │   double   │   double   │  double   │   double    │    double    │ int64  │   double   │     int64     │    int64     │         varchar         │     int32     │    date     │    double    │   varchar   │     varchar     │  varchar   │
├──────────┼─────────────┼─────────────┼────────────┼────────────┼────────────┼───────────┼─────────────┼──────────────┼────────┼─

In [ ]:
raw_data[raw_data['CHG_IN_OI'] > 2000000]

,INSTRUMENT,SYMBOL,EXPIRY_DT,STRIKE_PR,OPTION_TYP,OPEN,HIGH,LOW,CLOSE,SETTLE_PR,CONTRACTS,VAL_INLAKH,OPEN_INT,CHG_IN_OI,TIMESTAMP,exchange_id
35,FUTSTK,ASHOKLEY,29-Aug-2019,0.0,XX,69.85,73.40,67.20,69.25,69.25,13126.0,55080.69,80706000.0,5334000.0,01-AUG-2019,1
387,FUTSTK,SBIN,29-Aug-2019,0.0,XX,332.50,332.80,313.10,319.15,319.15,31914.0,308803.07,81147000.0,9354000.0,01-AUG-2019,1
465,FUTSTK,VEDL,29-Aug-2019,0.0,XX,152.40,152.40,144.45,146.30,146.30,8975.0,39712.11,73500000.0,2895000.0,01-AUG-2019,1
474,FUTSTK,YESBANK,29-Aug-2019,0.0,XX,88.95,91.60,85.90,88.80,88.80,35627.0,69774.70,122654400.0,3267000.0,01-AUG-2019,1
1634,OPTIDX,NIFTY,01-Aug-2019,11000.0,CE,64.70,73.25,0.05,0.55,0.00,1030257.0,8510273.14,4094400.0,3806775.0,01-AUG-2019,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2532956,OPTSTK,YESBANK,28-Nov-2019,85.0,CE,2.00,9.95,1.85,6.40,6.40,5243.0,10588.45,3363800.0,3119600.0,31-OCT-2019,1
2532957,OPTSTK,YESBANK,28-Nov-2019,90.0,CE,1.60,8.50,1.40,5.45,5.45,5872.0,12361.55,3066800.0,2424400.0,31-OCT-2019,1
2532959,OPTSTK,YESBANK,28-Nov-2019,100.0,CE,1.00,6.45,0.95,3.85,3.85,8955.0,20490.96,6615400.0,5088600.0,31-OCT-2019,1
2532972,OPTSTK,YESBANK,28-Nov-2019,60.0,PE,10.55,11.30,3.00,5.90,5.90,12549.0,18168.06,6677000.0,4921400.0,31-OCT-2019,1


In [ ]:
# 1. Top 10 symbols by open interest (OI) change across exchanges

In [ ]:
con.sql(""" select i.symbol, sum(change_in_oi )
from fact_trade f
left join exchange e on f.exchange_id =e.exchange_id
left join contracts c on c.contract_id = f.contract_id
left join instruments i on i.instrument_id = c.instrument_id
group by symbol
order by 2 desc
limit 10 """)

┌────────────┬───────────────────┐
│   symbol   │ sum(change_in_oi) │
│  varchar   │      int128       │
├────────────┼───────────────────┤
│ IDEA       │        1343664000 │
│ NIFTY      │         568363275 │
│ YESBANK    │         414031200 │
│ SBIN       │         294438000 │
│ BANKNIFTY  │         220297840 │
│ IDFCFIRSTB │         196680000 │
│ BHEL       │         161970200 │
│ ASHOKLEY   │         147232000 │
│ GMRINFRA   │         147150000 │
│ PNB        │         119256100 │
├────────────┴───────────────────┤
│ 10 rows              2 columns │
└────────────────────────────────┘

In [ ]:
## Average Close Price For Each Trade day for Nifty Options

In [ ]:
con.sql( """ WITH cte  AS (
    SELECT
        f.trade_date,
        i.symbol,
        AVG(f.close_price) AS avg_close_price
    FROM fact_trade f
    left join contracts c
      ON f.contract_id = c.contract_id
    left join instruments i
      ON c.instrument_id = i.instrument_id
    WHERE i.symbol = 'NIFTY'
      AND i.instrument_type = 'OPTIDX'
      AND f.close_price IS NOT NULL
    GROUP BY f.trade_date,i.symbol
)
select * from cte
""")

┌────────────┬─────────┬────────────────────┐
│ trade_date │ symbol  │  avg_close_price   │
│    date    │ varchar │       double       │
├────────────┼─────────┼────────────────────┤
│ 2019-10-03 │ NIFTY   │ 1043.1351311084643 │
│ 2019-09-03 │ NIFTY   │ 1023.4617933130716 │
│ 2019-09-04 │ NIFTY   │ 1021.3875303398078 │
│ 2019-09-06 │ NIFTY   │ 1031.6734144088687 │
│ 2019-11-07 │ NIFTY   │  1061.819313432837 │
│ 2019-10-09 │ NIFTY   │ 1052.9348621103136 │
│ 2019-09-13 │ NIFTY   │  1031.518555900623 │
│ 2019-09-19 │ NIFTY   │ 1024.9802964793093 │
│ 2019-08-21 │ NIFTY   │ 1019.2441078963246 │
│ 2019-10-23 │ NIFTY   │ 1064.7475241545915 │
│     ·      │   ·     │          ·         │
│     ·      │   ·     │          ·         │
│     ·      │   ·     │          ·         │
│ 2019-11-06 │ NIFTY   │  1062.746518230724 │
│ 2019-11-08 │ NIFTY   │  1068.103739445116 │
│ 2019-10-11 │ NIFTY   │ 1062.6583787878808 │
│ 2019-11-14 │ NIFTY   │ 1065.2064681107113 │
│ 2019-10-16 │ NIFTY   │ 1059.7783

In [ ]:
#Volatility analysis: 7-day rolling std dev of close prices for NIFTY options.

In [ ]:
con.sql( """ WITH cte  AS (
    SELECT
        f.trade_date,
        i.symbol,
        AVG(f.close_price) AS avg_close_price
    FROM fact_trade f
    left join contracts c
      ON f.contract_id = c.contract_id
    left join instruments i
      ON c.instrument_id = i.instrument_id
    WHERE i.symbol = 'NIFTY'
      AND i.instrument_type = 'OPTIDX'
      AND f.close_price IS NOT NULL
    GROUP BY f.trade_date,i.symbol
)
SELECT
    trade_date,
    avg_close_price,
    STDDEV_SAMP(avg_close_price) OVER (
        ORDER BY trade_date
        ROWS BETWEEN 6 PRECEDING AND CURRENT ROW
    ) AS rolling_7d_volatility
FROM cte
ORDER BY trade_date; """)

┌────────────┬────────────────────┬───────────────────────┐
│ trade_date │  avg_close_price   │ rolling_7d_volatility │
│    date    │       double       │        double         │
├────────────┼────────────────────┼───────────────────────┤
│ 2019-08-01 │ 1021.4163017396534 │                  NULL │
│ 2019-08-02 │ 1022.4495798319343 │    0.7306379459034106 │
│ 2019-08-05 │ 1020.2780312124863 │    1.0861972515486653 │
│ 2019-08-06 │  1015.134398939306 │    3.2469225694265287 │
│ 2019-08-07 │ 1014.6007660577508 │    3.6543233439637346 │
│ 2019-08-08 │ 1011.7579699469665 │     4.346449445354775 │
│ 2019-08-09 │ 1018.1186455847267 │    3.9724724107217124 │
│ 2019-08-13 │ 1016.6071811680583 │    3.6185712114985056 │
│ 2019-08-14 │ 1014.5179464285726 │    2.7664261263815604 │
│ 2019-08-16 │ 1021.8357509047061 │     3.208449546362414 │
│     ·      │          ·         │             ·         │
│     ·      │          ·         │             ·         │
│     ·      │          ·         │     

In [ ]:
#Option chain summary: Grouped by expiry_dt and strike_pr, calculating implied volume.

In [ ]:
 con.sql("""
 Select i.instrument_type,expiry_date, strike_price, sum(volume) as implied_volume,
 sum(case when c.option_type = 'CE' then f.volume else 0 end) as ce_volume,
 sum(case when c.option_type = 'PE' then f.volume else 0 end) as pe_volume
  from fact_trade f
  left join contracts c
      ON f.contract_id = c.contract_id
    left join instruments i
      ON c.instrument_id = i.instrument_id
    WHERE i.instrument_type in ('OPTIDX','OPTSTK')
    group by i.instrument_type,expiry_date, strike_price
    """)

┌─────────────────┬─────────────┬──────────────┬────────────────┬───────────┬───────────┐
│ instrument_type │ expiry_date │ strike_price │ implied_volume │ ce_volume │ pe_volume │
│     varchar     │    date     │    double    │     int128     │  int128   │  int128   │
├─────────────────┼─────────────┼──────────────┼────────────────┼───────────┼───────────┤
│ OPTIDX          │ 2019-08-01  │      27200.0 │            389 │         4 │       385 │
│ OPTIDX          │ 2019-08-01  │      27300.0 │            118 │         0 │       118 │
│ OPTIDX          │ 2019-08-01  │      27400.0 │           3079 │         8 │      3071 │
│ OPTIDX          │ 2019-08-01  │      27500.0 │           9372 │        56 │      9316 │
│ OPTIDX          │ 2019-08-01  │      28700.0 │        2523585 │   1786472 │    737113 │
│ OPTIDX          │ 2019-08-01  │      28800.0 │        2121019 │   1789480 │    331539 │
│ OPTIDX          │ 2019-08-01  │      29200.0 │        1211132 │   1202898 │      8234 │
│ OPTIDX  

In [ ]:
#Performance-optimized query for max volume in last 30 days using indexes/window functions.

con.sql("""
EXPLAIN ANALYZE

with daily_vol as

(
    SELECT
        f.trade_date,
        i.symbol,
        SUM(f.volume) AS daily_volume,
        ROW_NUMBER() OVER (PARTITION BY i.symbol ORDER BY SUM(f.volume) DESC) AS row_num
    FROM fact_trade f
    inner join contracts c ON f.contract_id = c.contract_id
    inner join instruments i ON c.instrument_id = i.instrument_id
    WHERE f.trade_date >= DATE '2019-11-15' - INTERVAL 30 DAY
    GROUP BY
        f.trade_date,
        i.symbol
)
Select trade_date,symbol,daily_volume
from daily_vol
where row_num = 1
""")

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql("""
select max(trade_date)
from fact_trade
""")

┌─────────────────┐
│ max(trade_date) │
│      date       │
├─────────────────┤
│ 2019-11-15      │
└─────────────────┘

In [ ]:
con.sql("""
EXPLAIN ANALYZE
with daily_vol as
(
    SELECT
        f.trade_date,
        i.symbol,
        SUM(f.volume) AS daily_volume,
        ROW_NUMBER() OVER (
            PARTITION BY i.symbol
            ORDER BY SUM(f.volume) DESC
        ) AS row_num
    FROM fact_trade f
    inner join contracts c ON f.contract_id = c.contract_id
    inner join instruments i ON c.instrument_id = i.instrument_id
    WHERE f.trade_date >= DATE '2019-11-15' - INTERVAL 30 DAY
    GROUP BY
        f.trade_date,
        i.symbol
)
Select trade_date,symbol,daily_volume
from daily_vol
where row_num = 1
""")


┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql("""
CREATE INDEX idx_fact_trade_date
ON fact_trade(trade_date);
""")

CatalogException: Catalog Error: Index with name "idx_fact_trade_date" already exists!

In [ ]:
con.sql("""
EXPLAIN ANALYZE
with daily_vol as
(
    SELECT
        f.trade_date,
        i.symbol,
        SUM(f.volume) AS daily_volume,
        ROW_NUMBER() OVER (
            PARTITION BY i.symbol
            ORDER BY SUM(f.volume) DESC
        ) AS row_num
    FROM fact_trade f
    inner join contracts c ON f.contract_id = c.contract_id
    inner join instruments i ON c.instrument_id = i.instrument_id
    WHERE f.trade_date >= DATE '2019-11-15' - INTERVAL 30 DAY
    GROUP BY
        f.trade_date,
        i.symbol
)
Select trade_date,symbol,daily_volume
from daily_vol
where row_num = 1
""")

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql("""
CREATE INDEX exchange_id
ON exchange(exchange_id);
""")

In [ ]:
con.sql("""
EXPLAIN ANALYZE
with daily_vol as
(
    SELECT
        f.trade_date,
        i.symbol,
        SUM(f.volume) AS daily_volume,
        ROW_NUMBER() OVER (
            PARTITION BY i.symbol
            ORDER BY SUM(f.volume) DESC
        ) AS row_num
    FROM fact_trade f
    inner join contracts c ON f.contract_id = c.contract_id
    inner join instruments i ON c.instrument_id = i.instrument_id
    WHERE f.trade_date >= DATE '2019-11-15' - INTERVAL 30 DAY
    GROUP BY
        f.trade_date,
        i.symbol
)
Select trade_date,symbol,daily_volume
from daily_vol
where row_num = 1
""")

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [ ]:
con.sql(""" DROP INDEX contract_id
""")


In [ ]:
con.sql("""
EXPLAIN ANALYZE
with daily_vol as
(
    SELECT
        f.trade_date,
        i.symbol,
        SUM(f.volume) AS daily_volume,
        ROW_NUMBER() OVER (
            PARTITION BY i.symbol
            ORDER BY SUM(f.volume) DESC
        ) AS row_num
    FROM fact_trade f
    inner join contracts c ON f.contract_id = c.contract_id
    inner join instruments i ON c.instrument_id = i.instrument_id
    WHERE f.trade_date >= DATE '2019-11-15' - INTERVAL 30 DAY
    GROUP BY
        f.trade_date,
        i.symbol
)
Select trade_date,symbol,daily_volume
from daily_vol
where row_num = 1
""")

┌───────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────